In [1]:
#Code origin
#Author: Alexander Valentini

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import AutoPeftModelForCausalLM
from datasets import load_dataset
import numpy as np
#Using quantization, but this should probably not be in the final version:
from pathlib import Path
import os

#FOR QUANTIZATION:

#bnb_config = BitsAndBytesConfig(
#    load_in_4bit=True,
#    llm_int8_threshold=6.0,
#    llm_int8_has_fp16_weight=False,
#    bnb_4bit_compute_dtype=torch.bfloat16,
#    bnb_4bit_use_double_quant=True,
#    bnb_4bit_quant_type="nf4",
#)

#model_name = 'AlexVal/dpo_model'

safetensors_path = 'models/mcqa_model_full_3B_sft_mcqa'

#FOR LOADING IN AND MERGING LORA ADAPTER:

#os.path.abspath(os.getcwd())
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#model = AutoPeftModelForCausalLM.from_pretrained(
#     safetensors_path,
#     torch_dtype=torch.float16,
#     attn_implementation="sdpa",

#).to(device)

#merged_model = model.merge_and_unload()
#merged_model.resize_token_embeddings(len(tokenizer))
#merged_model.save_pretrained('dpo_model_1.6B/merged_model',safe_serialization=True)


#LOADING IN TOKENIZER AND MODEL:
tokenizer = AutoTokenizer.from_pretrained("new_tokenizer")
#tokenizer = AutoTokenizer.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForCausalLM.from_pretrained(safetensors_path, torch_dtype=torch.float16, use_safetensors = True).to(device)
#model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16).to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
You have loaded an AWQ model on CPU and have a CUDA device available, make sure to set your model on a GPU device in order to run your model.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


In [1]:
#os.path.exists('D:\shared\DTU\10. Semester\Modern Natural Language Processing\Project\Milestone 3\github_project\project-m3-2024-dreamteam\model\mcqa_model')

In [2]:
model

StableLmForCausalLM(
  (model): StableLmModel(
    (embed_tokens): Embedding(50278, 2560)
    (layers): ModuleList(
      (0-31): 32 x StableLmDecoderLayer(
        (self_attn): StableLmSdpaAttention(
          (q_proj): WQLinear_GEMM(in_features=2560, out_features=2560, bias=False, w_bit=4, group_size=128)
          (k_proj): WQLinear_GEMM(in_features=2560, out_features=2560, bias=False, w_bit=4, group_size=128)
          (v_proj): WQLinear_GEMM(in_features=2560, out_features=2560, bias=False, w_bit=4, group_size=128)
          (o_proj): WQLinear_GEMM(in_features=2560, out_features=2560, bias=False, w_bit=4, group_size=128)
          (attention_dropout): Dropout(p=0.0, inplace=False)
          (rotary_emb): StableLmRotaryEmbedding()
        )
        (mlp): StableLmMLP(
          (gate_proj): WQLinear_GEMM(in_features=2560, out_features=6912, bias=False, w_bit=4, group_size=128)
          (up_proj): WQLinear_GEMM(in_features=2560, out_features=6912, bias=False, w_bit=4, group_size=128

In [21]:
#os.path.isdir('sft_stablelm_zephyr_3b_debug')

In [3]:
#model = merged_model
prompt = [{'role': 'user', 'content': 'List 3 synonyms for the word "tiny"'}]
inputs = tokenizer.apply_chat_template(
    prompt,
    add_generation_prompt=True,
    return_tensors='pt'
)

tokens = model.generate(
    inputs.to(model.device),
    max_new_tokens=50,
    temperature=0.8,
    do_sample=True,
     pad_token_id=tokenizer.pad_token_id
)

print(tokenizer.decode(tokens[0], skip_special_tokens=False))

d:\ProgramData\Anaconda3\envs\MNLP\lib\site-packages\transformers\models\stablelm\modeling_stablelm.py:485: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


<|user|>
List 3 synonyms for the word "tiny"<|endoftext|>
<|assistant|>
1.
2. Small
3. Petite<|endoftext|>


In [5]:
from huggingface_hub import login
login()

In [6]:
import huggingface_hub
model.push_to_hub(
 repo_id="mcqa_model_full_only_mcqa-awq_newest",
 commit_message="Commiting model"
)

model.safetensors:   0%|          | 0.00/1.83G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/AlexVal/mcqa_model_full_only_mcqa-awq_newest/commit/15faa6118b6a38da5120978b925aca15e2d5fcec', commit_message='Commiting model', commit_description='', oid='15faa6118b6a38da5120978b925aca15e2d5fcec', pr_url=None, pr_revision=None, pr_num=None)

In [7]:
tokenizer.push_to_hub(
 repo_id="mcqa_model_full_only_mcqa-awq_newest",
 commit_message="new_tokenizer"
)

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

d:\ProgramData\Anaconda3\envs\MNLP\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\alex\.cache\huggingface\hub\models--AlexVal--mcqa_model_full_only_mcqa-awq_newest. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


CommitInfo(commit_url='https://huggingface.co/AlexVal/mcqa_model_full_only_mcqa-awq_newest/commit/258fba6a3e0eb9d000c8c7d339ac45460b6e0361', commit_message='new_tokenizer', commit_description='', oid='258fba6a3e0eb9d000c8c7d339ac45460b6e0361', pr_url=None, pr_revision=None, pr_num=None)

In [8]:
tokenizer.model_max_length

2048